# 🍌 一起在 Discord 玩 AI 奈米香蕉 🍌

<br>[![Licence](https://img.shields.io/github/license/Ileriayo/markdown-badges?style=for-the-badge)](https://opensource.org/license/MIT)
[![Nano Banana](https://img.shields.io/badge/Nano%20Banana-yellow?style=for-the-badge&logo=gamebanana&logoColor=white)](https://ai.dev/models/gemini-2-5-flash-image)
[![Google Gemini](https://img.shields.io/badge/gemini-8E75B2?style=for-the-badge&logo=google%20gemini&logoColor=white)](http://gemini.google/)


我們將引導您在 Google Colab 上設置並運行 [GDG-NTUST course-1141-9 專案](https://github.com/GDG-NTUST/course-1141-9)
- 使用指令生成圖像
- 啟動 Discord Bot 讓使用者透過 Discord 介面生成圖像

## 📋 前置需求
- Google API Key 可以從 [Google AI Studio - API keys](https://ai.dev/app/api-keys) 取得
- Discord Bot Token 可以從 [Discord 開發者平臺](https://discord.com/developers/applications) Application Bot 取得
- Discord Guild ID 可以從 Discord 伺服器設定頁面取得

## 📥 步驟 1: Clone 專案

In [ ]:
!git clone https://github.com/GDG-NTUST/course-1141-9.git
%cd course-1141-9

## 📦 步驟 2: 安裝 uv 套件管理工具

### 📦 步驟 2-1: 安裝 uv

In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

### 📦 步驟 2-2: 將 uv 加入 PATH

In [ ]:
import os

os.environ['PATH'] = f'/root/.cargo/bin:{os.environ["PATH"]}'

### 📦 步驟 2-3: 安裝相依套件

In [ ]:
!uv sync

## 🔑 步驟 3: 設定環境變數

請在下方設定環境變數：
- GOOGLE_API_KEY: 執行程式碼後即可輸入 API Key
- SYSTEM_PROMPT: 用於指導 AI 生成圖像的系統提示
- LOG_LEVEL: 設定日誌等級（debug、info、warning、error、critical）
- MODEL_NAME: 指定使用的模型名稱（`gemini-2.5-flash-image`、`gemini-3-pro-image-preview`）

In [ ]:
from getpass import getpass
from pathlib import Path

GOOGLE_API_KEY: str = getpass('請輸入 Google API Key: ')
SYSTEM_PROMPT = 'You are a helpful assistant that generates images based on user prompts. Always in "Minecraft" style.'
LOG_LEVEL: str = 'info'
MODEL_NAME: str = 'gemini-2.5-flash-image'

# 建立 .env 檔案
with Path('.env').open('w') as f:
    f.write(
        f'GOOGLE_API_KEY={GOOGLE_API_KEY}\n'
        f'SYSTEM_PROMPT={SYSTEM_PROMPT}\n'
        f'LOG_LEVEL={LOG_LEVEL}\n'
        f'MODEL_NAME={MODEL_NAME}\n',
    )

## 🎨 步驟 4: 生成圖像

### 📁 步驟 4-1: 前往 `src`

In [ ]:
%cd src/

### 📜 步驟 4-2: 執行生成圖像指令

可以在 `-p` 後面的雙引號內修改提示詞來生成不同的圖像

In [ ]:
!uv run python -m nano_banana.api.demo.demo -p "A dog is playing a game"

#### 📎 可用參數

| 短旗 | 長旗 | 必需 | 說明 |
| ---- | ---- | ---- | ---- |
| `-p` | `--prompt` | ✅ 是 | 圖像生成或編輯的提示詞 |
| `-i` | `--image` | ❌ 否 | 輸入圖像的路徑（可多個） |
| `-k` | `--key` | ❌ 否 | Google API Key（覆蓋環境變數） |

#### ✔️ 範例

```shell
# 文字轉圖像
!uv run python -m nano_banana.api.demo.demo -p "A cat is sleeping"

# 文字和圖像轉圖像
!uv run python -m nano_banana.api.demo.demo -p "Have dinner together" -i image1.png image2.png

# 自訂 API Key
!uv run python -m nano_banana.api.demo.demo -p "A pig is watching the sunrise." -k "your-api-key"
```

#### 👉 提醒
- 程式執行時的目錄需要在 `src`
 
- [圖像生成的建議與限制](https://ai.google.dev/gemini-api/docs/image-generation?hl=zh-tw#limitations)

  - 建議使用英文提示詞
  > 為獲得最佳成效，請使用下列語言：英文、ar-EG、de-DE、es-MX、fr-FR、hi-IN、id-ID、it-IT、ja-JP、ko-KR、pt-BR、ru-RU、ua-UA、vi-VN、zh-CN。

  - 輸入圖像的數量有限制
  > gemini-2.5-flash-image 最多可輸入 3 張圖片，gemini-3-pro-image-preview 則支援 5 張高保真度圖片，最多可輸入 14 張圖片。

- [提示撰寫指南和策略](https://ai.google.dev/gemini-api/docs/image-generation?hl=zh-tw#prompt-guide)

### 顯示生成的圖像

生成的圖像會儲存在 `nano_banana/api/demo/outputs/` 目錄中，顯示最新生成的圖像：

In [ ]:
import logging
import os
from pathlib import Path

from IPython.display import Image

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

output_dir = Path('nano_banana/api/demo/outputs')
if output_dir.exists():
    image_files = sorted(output_dir.glob('*.png'), key=os.path.getmtime, reverse=True)
    if image_files:
        latest_image = image_files[0]
        display(Image(filename=str(latest_image)))
    else:
        logger.error('❌ 找不到生成的圖像')
else:
    logger.error('❌ outputs 目錄不存在')

## 🗝️ 步驟 5: 設定 Discord Bot 的環境變數

In [ ]:
from getpass import getpass
from pathlib import Path

DISCORD_TOKEN: str = getpass('請輸入 Discord Token: ')
DISCORD_GUILD_ID: str = getpass('請輸入 Discord Guild ID: ')

# 將 Discord 環境變數加到 .env 檔案
with Path('.env').open('a') as f:
    f.write(
        f'DISCORD_TOKEN={DISCORD_TOKEN}\n'
        f'DISCORD_GUILD_ID={DISCORD_GUILD_ID}\n',
    )

## 🤖 步驟 6: 啟動 Discord Bot

啟動後，即可在 Discord 伺服器中的任意頻道使用指令來生成圖像，Bot 會在當前的頻道回傳生成的圖像。

👉 **提醒**：Colab 最長可執行 12 小時

In [ ]:
!uv run python -m nano_banana.main

### Discord 命令

#### 斜線命令

`/畫圖 prompt: <提示詞>`

**範例**：`/畫圖 prompt: 一隻正在玩電玩的狗`

#### 訊息型（須指定伺服器）

- 發送一條文字訊息，可選擇附上圖像
- 根據文字（與圖像）生成新圖像

## 📚 其他資訊

### 疑難排解
- `ValueError: GOOGLE_API_KEY must be set in environment variables.`
  - 請重新執行 [🔑 步驟 3: 設定環境變數]，確認已正確設定 GOOGLE_API_KEY 環境變數
- `Unsupported MODEL_NAME:`
  - 請確認 MODEL_NAME 是否設置為支援的模型名稱（`gemini-2.5-flash-image`、`gemini-3-pro-image-preview`）
- `ValueError: Prompt is required.`
  - 請確認在生成圖像時是否有提供提示詞（系統提示或提示詞至少要有 1 個字）
- `Empty response from Gemini model.`
  - 未收到 Gemini API 回應，請再試一次
- `Number of input images exceeds the maximum of {settings.MAX_IMAGE_PER_REQUEST}.`
  - 輸入的圖像數量超過模型限制，請減少輸入圖像數量
- `Cannot fetch reference message:`
  - 請確認 Discord 的訊息是否存在且可被存取

### 相關連結
- [GDG-NTUST course-1141-9 專案 GitHub](https://github.com/GDG-NTUST/course-1141-9)
- [GDG-NTUST](https://gdg-ntust.org)